In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import src.names_numbers as NN
from src.names_numbers import heroes as H 

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from src.roc import plot_roc
from sklearn.metrics import confusion_matrix, mean_squared_error
plt.style.use('ggplot')

In [3]:
df = pd.read_csv('data/dota2Train.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.rename(columns={0: 'winner',
                   1: 'region',
                   2: 'mode',
                   3: 'type'}, inplace=True)
df.head()

,winner,region,mode,type,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.columns = [(i-3) if type(i) == int else i for i in df.columns]

In [6]:
NN.get_names(df)

In [7]:
df.head()

,winner,region,mode,type,antimage,axe,bane,bloodseeker,crystal_maiden,drow_ranger,...,legion_commander,techies,ember_spirit,earth_spirit,abyssal_underlord,terrorblade,phoenix,oracle,winter_wyvern,arc_warden
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
NN.get_nums(df)

In [9]:
df.head()

,winner,region,mode,type,1,2,3,4,5,6,...,104,105,106,107,108,109,110,111,112,113
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#start random forest work need binary
# winner column 0 = team B wins,  1 = team A
rfdf = df.copy()
for i in ['region', 'mode', 'type']:
    rfdf.pop(i)
winner = rfdf.pop('winner')
y = winner.replace(-1, 0)
#make X values binary, each column is char-team, 1 denotes picked.
X = rfdf
X2 = X.copy()
X.columns = [(str(i) + 'A') for i in X.columns]
X2.columns = [(str(i) + 'B') for i in X2.columns]
X.replace(-1, 0, inplace=True)
X2.replace(1, 0, inplace=True)
X2.replace(-1, 1, inplace=True)
X = pd.concat([X, X2], axis=1)
X.shape

(92650, 226)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print("\nscore:", rf.score(X_test, y_test))
y_predict = rf.predict(X_test)
print("\nconfusion matrix:")
print(confusion_matrix(y_test, y_predict))
print("\nprecision:", precision_score(y_test, y_predict))
print("    recall:", recall_score(y_test, y_predict))


score: 0.5772136597159263

confusion matrix:
[[5464 5469]
 [4324 7906]]

precision: 0.5911028037383178
    recall: 0.646443172526574


In [12]:
rf2 = RandomForestClassifier()

In [13]:
#XGB testing
import xgboost as xgb

In [14]:
# y = digits['target']
# X = digits['data']
# kf = KFold(n_splits=2, shuffle=True, random_state=rng)
# for train_index, test_index in kf.split(X):
#     xgb_model = xgb.XGBClassifier().fit(X[train_index], y[train_index])
#     predictions = xgb_model.predict(X[test_index])
#     actuals = y[test_index]
#     print(confusion_matrix(actuals, predictions))

In [41]:
#X.reset_index(inplace=True)
kf = KFold(n_splits=2, shuffle=True)
for train_index, test_index in kf.split(X[:20]):
    xgb_model = xgb.XGBClassifier().fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(confusion_matrix(actuals, predictions))
    break

0            0
1            1
2            2
3            3
4            4
         ...  
92645    92645
92646    92646
92647    92647
92648    92648
92649    92649
Name: index, Length: 92650, dtype: int64


ValueError: Please reshape the input data X into 2-dimensional matrix.

In [22]:
len(train_index)

74120

In [43]:
len(X.index)

92650

In [30]:
X.to_pickle('x.pkl.bz2', compression='bz2')

In [42]:
X.head()

0    0
1    1
2    2
3    3
4    4
Name: index, dtype: int64